### CCR (Primal)

#### Objective function:
$\max 700u_1 + 300u_2$

s.t.

$700u1 + 300u2 − 40v1 − 500v2 \leqslant 0$ (EFF. DMU1)

$300u1 + 600u2 − 50v1 − 500v2 \leqslant  0$ (EFF. DMU2)

$200u1 + 700u2 − 50v1 − 400v2 \leqslant  0$ (EFF. DMU3)

$400u1 + 600u2 − 50v1 − 500v2 \leqslant  0$ (EFF. DMU4)

$500u1 + 400u2 − 40v1 − 400v2 \leqslant  0$ (EFF. DMU5)

$500u1 + 500u2 − 50v1 − 500v2 \leqslant 0$ (EFF. DMU6)

$800u1 + 500u2 − 40v1 − 600v2 \leqslant  0$ (EFF. DMU7)

$300u1 + 200u2 − 30v1 − 400v2 \leqslant 0$ (EFF. DMU8)

$40v1 + 500v2 = 1$ (Norm inputs)

$v1 \geqslant 0.00001$ (non-negativity $v_1$)

$v2 \geqslant 0.00001$ (non-negativity $v_2$)

$u1 \geqslant 0.00001$ (non-negativity $u_1$)

$u2 \geqslant 0.00001$ (non-negativity $u_2$)

In [ ]:
import gurobipy as gu

ccr = gu.Model("DEA")

# Variables
v1 = ccr.addVar(lb=0.00001, name='v1')
v2 = ccr.addVar(lb=0.00001, name='v2')
u1 = ccr.addVar(lb=0.00001, name='u1')
u2 = ccr.addVar(lb=0.00001, name='u2')

# Objective
ccr.setObjective(300*u1 + 600*u2, gu.GRB.MAXIMIZE)

# Constraints
constraints = [
    ccr.addLConstr(700*u1 + 300*u2 - 40*v1 - 500*v2 <= 0, name="EFF. DMU1"),
    ccr.addLConstr(300*u1 + 600*u2 - 50*v1 - 500*v2 <= 0, name="EFF. DMU2"),
    ccr.addLConstr(200*u1 + 700*u2 - 50*v1 - 400*v2 <= 0, name="EFF. DMU3"),
    ccr.addLConstr(400*u1 + 600*u2 - 50*v1 - 500*v2 <= 0, name="EFF. DMU4"),
    ccr.addLConstr(500*u1 + 400*u2 - 40*v1 - 400*v2 <= 0, name="EFF. DMU5"),
    ccr.addLConstr(500*u1 + 500*u2 - 50*v1 - 500*v2 <= 0, name="EFF. DMU6"),
    ccr.addLConstr(800*u1 + 500*u2 - 40*v1 - 600*v2 <= 0, name="EFF. DMU7"),
    ccr.addLConstr(300*u1 + 200*u2 - 30*v1 - 400*v2 <= 0, name="EFF. DMU8"),
    ccr.addLConstr(50 * v1 + 500 * v2 == 1, name="Norm. Input")
]

ccr.optimize()

if ccr.Status == gu.GRB.OPTIMAL:
    print(f"Objective value: {ccr.ObjVal}")
    print(f"v1: {v1.X}")
    print(f"v2: {v2.X}")
    print(f"u1: {u1.X}")
    print(f"u2: {u2.X}")

    for constr in ccr.getConstrs():
        constr_value = ccr.getRow(constr).getValue()
        dual_value = constr.getAttr(gu.GRB.Attr.Pi)
        print(f"{constr.ConstrName}: Obj-Value = {round(constr_value,5)}, Shadow Price = {round(dual_value,5)}")
else:
    print("No optimal solution found.")

### CCR (Dual)

#### Objective function
$\min \theta_0$

s.t.


$\theta_0 \cdot 40 \geqslant 40\lambda_1 + 50\lambda_2 + 50\lambda_3 + 50\lambda_4 + 40\lambda_5 + 50\lambda_6 + 40\lambda_7 + 30\lambda_8$ (Input 1)

$\theta_0 \cdot 500 \geqslant 500\lambda_1 + 500\lambda_2 + 400\lambda_3 + 500\lambda_4 + 400\lambda_5 + 500\lambda_6 + 600\lambda_7 + 400\lambda_8 $(Input 2)

$700 \leqslant 700\lambda_1 + 300\lambda_2 + 200\lambda_3 + 400\lambda_4 + 500\lambda_5 + 500\lambda_6 + 800\lambda_7 + 300\lambda_8$ (Output 1)

$300 \leqslant 300\lambda_1 + 600\lambda_2 + 700\lambda_3 + 600\lambda_4 + 400\lambda_5 + 500\lambda_6 + 500\lambda_7 + 200\lambda_8$ (Output 2)

$\lambda_j \geqslant 0\quad \forall j$

In [3]:
## CCR Dual

dual = gu.Model("DEA")

# Variables
lmb1 = dual.addVar(name='lmb1')
lmb2 = dual.addVar(name='lmb2')
lmb3 = dual.addVar(name='lmb3')
lmb4 = dual.addVar(name='lmb4')
lmb5 = dual.addVar(name='lmb5')
lmb6 = dual.addVar(name='lmb6')
lmb7 = dual.addVar(name='lmb7')
lmb8 = dual.addVar(name='lmb8')
theta0 = dual.addVar(name='theta0')

constraints1 = [
dual.addLConstr(theta0*40 >= 40*lmb1 + 50*lmb2 + 50*lmb3 + 50*lmb4 + 40*lmb5 + 50*lmb6 + 40*lmb7 + 30*lmb8, name = 'Input 1'),
dual.addLConstr(theta0*500 >= 500*lmb1 + 500*lmb2 + 400*lmb3 + 500*lmb4 + 400*lmb5 + 500*lmb6 + 600*lmb7 + 400*lmb8, name = 'Input 2'),
dual.addLConstr(700 <= 700*lmb1 + 300*lmb2 + 200*lmb3 + 400*lmb4 + 500*lmb5 + 500*lmb6 + 800*lmb7 + 300*lmb8, name = 'Output 1'),
dual.addLConstr(300 <= 300*lmb1 + 600*lmb2 + 700*lmb3 + 600*lmb4 + 400*lmb5 + 500*lmb6 + 500*lmb7 + 200*lmb8, name = 'Output 2')
]

dual.setObjective(theta0, gu.GRB.MINIMIZE)
dual.optimize()
if dual.Status == gu.GRB.OPTIMAL:
    print(f"Objective value: {dual.ObjVal}")
    print(f"v1: {v1.X}")
    print(f"v2: {v2.X}")
    print(f"u1: {u1.X}")
    print(f"u2: {u2.X}")

    for constr in dual.getConstrs():
        constr_value = dual.getRow(constr).getValue()
        dual_value = constr.getAttr(gu.GRB.Attr.Pi)
        print(f"{constr.ConstrName}: Obj-Value = {round(constr_value,5)}, Shadow Price = {round(dual_value,5)}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 9 columns and 34 nonzeros
Model fingerprint: 0x95fffaac
Coefficient statistics:
  Matrix range     [3e+01, 8e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 7e+02]
Presolve time: 0.00s
Presolved: 4 rows, 9 columns, 34 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.125000e+01   0.000000e+00      0s
       3    1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.000000000e+00
Objective value: 1.0
v1: 0.0199
v2: 1e-05
u1: 0.00013456521739130432
u2: 0.001388695652173913
Input 1: Obj-Value = 0.0, Shadow Price = 0.00714
Input 2: Obj-Value = 0.0, Shadow Price = 0.00143
Output 1: Obj-Value = 700.0, Shadow Price = 0.00143
